In [110]:
%reset -f

In [25]:
# 79-character line limit
######### ######### ######### ######### ######### ######### ######### #########
import numpy as np # The NumPy library
#import math # The math module, np includes it
from scipy.integrate import quad # Method for integration in scipy.integrate sub-package
from scipy.optimize import linprog


cells_num = 8

# The sizes are for UQs for number of cells: 4, 8, 16, 32, 64, 128, 256.

# The opitimized spaces for the Guassian distribution in a uniform quantizer(UQ).
all_size_gaus = np.array([0.9957, 0.5860, 0.3352, 0.1881, 0.1041, 0.0569, 0.0308])

# The opitimized spaces for the Laplacian distribution in a uniform quantizer(UQ).
all_size_laplacian = np.array(
                   [1.414, 1.0873, 0.8707, 0.7309, 0.6334, 0.5613, 0.5055])


'''The opitimized spaces for a Mixed Gaussian distribution in 
a uniform quantizer(UQ). This distribution is not symmetric and the values are
for for cells number 2, 4, 8, 16, 32, 64, 128, 256'''
all_boudary1s = [-0.05, -2.02, -3.17, -3.93, -4.49, -5.08, -5.16, -5.50]
all_deltas = [4, 1.81, 1, 0.54, 0.29, 0.16, 0.08, 0.05]

'''To change the symmetric pdf, two parts of code must be changed include the 
cell_size and pdf'''

cell_size = all_size_gaus[int(np.log2(cells_num))-2]
# cell_size = all_size_laplacian[int(np.log2(cells_num))-2]

# np.arange has rounding error issue.
# boundaries_symm = np.arange(-(cells_num/2-1) * cell_size, # For symmetric distributions
#                              (cells_num/2) * cell_size,
#                             cell_size)

# Boundaries for the symmetric distributions are computed.
boundaries_symm = np.linspace(-(cells_num/2-1) * cell_size, # For symmetric distributions
                               (cells_num/2-1) * cell_size,
                              cells_num-1)

# Boundaries for the asymmetric distributions are computed.
boundary1 = all_boudary1s[int(np.log2(cells_num))-1]
delta = all_deltas[int(np.log2(cells_num))-1]

boundaries_asymm = []
for i in range (0,cells_num-1):
        boundary = boundary1 + (i) * delta
        boundaries_asymm = np.append(boundaries_asymm, boundary)
        
'''To change the symmetric pdf to asymetric pdf, two parts of code 
must be changed include the boundaries and pdf'''        

boundaries = boundaries_symm # For symmetric distributions
# boundaries = boundaries_asymm # For asymetric distributions
        

n_inf = float("-inf")
p_inf = float("inf")

boundaries = np.insert(boundaries, 0, n_inf)
boundaries = np.append(boundaries, p_inf)





def pdf(x): # Defining the distribution
    gaus_std = 1
    gaus_mean = 0
    pdf = 1/(gaus_std*np.sqrt(2*np.pi)) * \
                  np.exp(-0.5*((x-gaus_mean)/gaus_std)**2) # Gaussian pdf
    
#     div = 1
#     pdf = 1/(2*div)*np.exp(-abs(x)/div) # Laplacian pdf

#     Mean1 = -2
#     Mean2 = 2
#     stdev1 = 1
#     stdev2 = 1
#     alpha = 0.7
#     pdf = (alpha)*(1/np.sqrt(2*np.pi*stdev1)*np.exp(-(x-Mean1)**2/(2*stdev1**2)))\
#         + (1-alpha)*(1/np.sqrt(2*np.pi*stdev2)*np.exp(-(x-Mean2)**2/(2*stdev2**2)))
    
    return pdf

def xpdf(x):
    xpdf = x * pdf(x)
    return xpdf

def x2pdf (x):
    x2pdf = x * xpdf(x)
    return x2pdf


prbs = []
xprbs = []
x2prbs = []
cell_reps = []
for i in range (0, cells_num):
    cell_prb, integ_err = quad(pdf, boundaries[i], boundaries[i+1])
    prbs = np.append(prbs, cell_prb)
    
    cell_xprb, integ_err = quad(xpdf, boundaries[i], boundaries[i+1])
    xprbs = np.append(xprbs, cell_xprb)
    
    cell_x2prb, integ_err = quad(x2pdf, boundaries[i], boundaries[i+1])
    x2prbs = np.append(x2prbs, cell_x2prb)
    
    cell_rep = cell_xprb / cell_prb
    cell_reps = np.append(cell_reps, cell_rep)
    

# Reconstruction value of each cell in a uniform quantizer
ddd = (boundaries[2]-boundaries[1])/2
all_ys = np.zeros(cells_num)
all_ys[0] = boundaries[1] - ddd

for i in range(1, cells_num):
    all_ys[i] = all_ys[i-1] + ddd*2


all_yijs = []
all_dijs = []
all_prb0s = []
for i in range(0,cells_num-1):
    for j in range(i+1,cells_num):
        
        '''Choose one Strategy'''
# Strategy1: Eve knows the source statistics which is rare in image encrytion
# Strategy2: Eve knows the reconstruction value of each cell
# Strategy3: Eve is not aware that encryption took place

#         possible_yij = (xprbs[i] + xprbs[j])/(prbs[i] + prbs[j]) # Strategy1
        
#         yi = xprbs[i]/prbs[i] # Strategy2, 3
#         yj = xprbs[j]/prbs[j] # Strategy2, 3

# Based on the uniform quantizer
        yi = all_ys[i] # Strategy2, 3
        yj = all_ys[j] # Strategy2, 3
        
#         possible_yij = (yi + yj)/2 # Strategy2
        
        
        all_yijs = np.append(all_yijs, possible_yij)
               
        
#         possible_dij\
#         = x2prbs[i] + possible_yij**2 * prbs[i] - possible_yij *2 * xprbs[i]\
#         + x2prbs[j] + possible_yij**2 * prbs[j] - possible_yij *2 * xprbs[j]
#         all_dijs = np.append(all_dijs, possible_dij) # Strategy1, 2
        
        possible_dij = 0.5*(
          x2prbs[i] + yi**2 * prbs[i] - yi *2 * xprbs[i]\
        + x2prbs[j] + yi**2 * prbs[j] - yi *2 * xprbs[j]
        + x2prbs[i] + yj**2 * prbs[i] - yj *2 * xprbs[i]\
        + x2prbs[j] + yj**2 * prbs[j] - yj *2 * xprbs[j])
        all_dijs = np.append(all_dijs, possible_dij) # Strategy3
        
                
        possible_prb0 = max(prbs[j],prbs[i])
        all_prb0s = np.append(all_prb0s, possible_prb0)

In [5]:
''' For just distortion, we implement the linear programming.'''

obj_coef = -all_dijs
all_length = int(cells_num * (cells_num-1) / 2)

A_eq = np.zeros([cells_num, all_length])
k = 0
for i in range(0, cells_num-1):
    for j in range(i+1,cells_num):
        A_eq[i,k]=1
        k+=1
k = 0
for j in range(1, cells_num):
    for i in range (j,cells_num):
        A_eq[i,k]=1
        k+=1

b_eq = np.ones([cells_num,])

result = linprog(obj_coef, A_eq=A_eq, b_eq=b_eq)
opt_xs = result.x
opt_pairs = np.round(result.x) # Pairs
opt_distortion = -result.fun # Value of the distortion

k = 0
xx = np.zeros([cells_num, cells_num])
for i in range(0,cells_num):
    for j in range(i+1,cells_num):
        xx[i,j] = opt_pairs[k]
        k +=1
        
lp_pairs_show = np.array(np.where(xx == 1))
opt_db = 10 * np.log10(opt_distortion)

print(lp_pairs_show)         
    
# print(opt_distortion)
print(opt_db)


# print(opt_xs)
# print(opt_pairs)
# print('\n', pairs)

[[0 1]
 [2 3]]
0.4041681850184491


In [26]:
'''For entropy and distortion, we implement the linear programming.'''

Omega = 1
Lambda = 0

obj_coef = -Omega * all_dijs - Lambda * all_prb0s
all_length = int(cells_num * (cells_num-1) / 2)

A_eq = np.zeros([cells_num, all_length])
k = 0
for i in range(0, cells_num-1):
    for j in range(i+1,cells_num):
        A_eq[i,k]=1
        k+=1
k = 0
for j in range(1, cells_num):
    for i in range (j,cells_num):
        A_eq[i,k]=1
        k+=1

b_eq = np.ones([cells_num,])

result = linprog(obj_coef, A_eq=A_eq, b_eq=b_eq)
opt_xs = result.x
opt_pairs = np.round(result.x) # Pairs
opt_distortion = -result.fun # Value of the distortion


k = 0
xx = np.zeros([cells_num, cells_num])
for i in range(0,cells_num):
    for j in range(i+1,cells_num):
        xx[i,j] = opt_pairs[k]
        k +=1
        
lp_pairs_show = np.array(np.where(xx == 1))
print(lp_pairs_show) 


opt_dijs = opt_pairs * all_dijs
opt_distortion = sum(opt_dijs)
opt_db = 10 * np.log10(opt_distortion)

opt_prb0s = opt_pairs * all_prb0s 
opt_prb0 = sum(opt_prb0s)
opt_entropy = -opt_prb0 * np.log2(opt_prb0)-(1-opt_prb0)*np.log2(1-opt_prb0)

    
print(opt_db)
print(opt_entropy)
# print(opt_xs)
# print(opt_pairs)
# print('\n', pairs)

[[0 1 2 3]
 [4 5 6 7]]
4.412704429648265
0.7970252813698468


In [100]:
# NBIA

nb_all_yijs = []
nb_all_dijs = []
power = np.log2(cells_num)
for i in range(0,int(cells_num/2)):
    j = int(cells_num/2 + i)
    print(i,j)
    nb_yij = (xprbs[i]+xprbs[j])/ (prbs[i]+prbs[j]) 
    nb_all_yijs = np.append(nb_all_yijs, nb_yij)
    nb_dij = x2prbs[i] + nb_yij**2 * prbs[i] - nb_yij *2 * xprbs[i]\
           + x2prbs[j] + nb_yij**2 * prbs[j] - nb_yij *2 * xprbs[j]
    nb_all_dijs = np.append(nb_all_dijs, nb_dij)

nb_distortion = np.sum(nb_all_dijs)
nb_db = 10 * np.log10(nb_distortion)

nb_prb0s = prbs[0: int(cells_num/2)]  
nb_prb0 = sum(nb_prb0s)
nb_entropy = -nb_prb0*np.log2(nb_prb0)-(1-nb_prb0)*np.log2(1-nb_prb0)

print('\n',nb_distortion)
print(nb_entropy)
print(nb_db)

0 16
1 17
2 18
3 19
4 20
5 21
6 22
7 23
8 24
9 25
10 26
11 27
12 28
13 29
14 30
15 31

 0.8921503978636394
1.0
-0.4956192650232582


In [92]:
nb_prb0s

array([0.15969797, 0.34030203])